In [1]:
from stock_bot.TradingSystem import *
from stock_bot.AlpacaDataManager import *
from stock_bot.DataFrameBacktest import *
from stock_bot.TA_LIB_FunctionMapping import *
from pathlib import Path


In [2]:
data_folder = Path('./config')
yaml_file = 'api_keys.yaml'
strategy  = 'ta-lib_example.yaml'
yaml_path = data_folder / yaml_file
strat_file = data_folder / strategy

with open(yaml_path, 'r') as file:
    yaml_config = yaml.safe_load(file)

with open("./config/ta-lib_example.yaml", 'r') as file:
    yaml_trade_config = yaml.safe_load(file)

api_key=yaml_config['api_key_paper']
api_secret=yaml_config['api_secret_paper']
data_fetcher = AlpacaDataFetcher(api_key, api_secret)


# Fetch historical data
historical_data = data_fetcher.get_historical_data(
    symbol=yaml_trade_config['symbol'],
    timeframe=yaml_trade_config["timeframe"],
    start_date=datetime(2024, 10, 1, tzinfo=pytz.UTC),
    end_date=datetime(2024, 12, 29, tzinfo=pytz.UTC)
)

    

historical_data = historical_data.droplevel('symbol')
historical_data.tail()

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2024-06-14 14:00:00+00:00,1.0004,1.02,0.980,0.9999,2122156.0,8222.0,0.996403
2024-06-17 08:00:00+00:00,1.0000,1.06,0.979,1.0500,3712913.0,7823.0,1.019573
2024-06-17 19:00:00+00:00,1.0410,1.05,1.010,1.0100,519445.0,1732.0,1.032681
2024-06-18 06:00:00+00:00,1.0400,1.04,1.010,1.0200,898762.0,3196.0,1.018814
2024-06-18 17:00:00+00:00,1.0100,1.02,0.980,0.9800,1362826.0,4238.0,0.992241


In [3]:
yaml_trade_config.get('entry_conditions')

[{'indicator': 'RSI', 'comparison': 'below', 'value': 40},
 {'indicator': 'BBANDS', 'comparison': 'crosses_below', 'value': 'lowerband'}]

In [4]:
# Initialize indicators
indicators = TALibIndicators()

# Calculate indicators for your DataFrame
def calculate_indicators(df: pd.DataFrame, config: Dict) -> pd.DataFrame:
    result_df = df.copy()
    
    for ind_config in config['indicators']:
        name = ind_config['name']
        params = ind_config.get('params', {})
        
        # Calculate indicator
        indicator_df = indicators.calculate_indicator(name, df, params)

        # Merge results
        result_df = pd.concat([result_df, indicator_df], axis=1)
    
    return result_df

# Use in your backtest
df = calculate_indicators(historical_data, yaml_trade_config)
df.head()

,open,high,low,close,volume,trade_count,vwap,rsi,upperband,middleband,lowerband,atr
timestamp,,,,,,,,,,,,
2024-01-02 01:00:00+00:00,1.0000,1.0100,1.0000,1.0000,5104.0,23.0,1.004066,NaN,NaN,NaN,NaN,NaN
2024-01-02 12:00:00+00:00,1.0000,1.0100,0.9130,0.9299,2851288.0,7163.0,0.957736,NaN,NaN,NaN,NaN,NaN
2024-01-02 23:00:00+00:00,0.9203,0.9390,0.9100,0.9350,58950.0,89.0,0.919915,NaN,NaN,NaN,NaN,NaN
2024-01-03 10:00:00+00:00,0.9353,0.9747,0.9043,0.9680,2822694.0,6859.0,0.941623,NaN,NaN,NaN,NaN,NaN
2024-01-03 21:00:00+00:00,0.9688,0.9800,0.9500,0.9800,67576.0,42.0,0.972386,NaN,NaN,NaN,NaN,NaN


In [5]:
dataframe_backtester = DataFrameBacktester(strat_file)
results = dataframe_backtester.backtest(df)


{'symbol': 'RGTI', 'timeframe': '1h', 'mode': 'backtest', 'risk_management': {'position_sizing_method': 'atr_based', 'risk_per_trade': 0.1, 'max_position_size': 1000, 'stop_loss': 0.05, 'take_profit': 0.1, 'max_drawdown': 0.15, 'atr_multiplier': 2.0}, 'indicators': [{'name': 'RSI', 'params': {'timeperiod': 14}}, {'name': 'BBANDS', 'params': {'timeperiod': 20, 'nbdevup': 1.5, 'nbdevdn': 1.5, 'matype': 0}}, {'name': 'ATR', 'params': {'timeperiod': 14}}], 'entry_conditions': [{'indicator': 'RSI', 'comparison': 'below', 'value': 40}, {'indicator': 'BBANDS', 'comparison': 'crosses_below', 'value': 'lowerband'}], 'exit_conditions': [{'indicator': 'RSI', 'comparison': 'above', 'value': 75}, {'indicator': 'ATR', 'comparison': 'above', 'value': 10}]}


1. indicator:  RSI
2. comparison:  below
3. Value:  40


1. indicator:  RSI
2. comparison:  below
3. Value:  40


1. indicator:  RSI
2. comparison:  below
3. Value:  40


1. indicator:  RSI
2. comparison:  below
3. Value:  40


1. indicator:  RSI

/Users/frederickmacdonald/Documents/Projects/true_botist/true-bautist/stock_bot/DataFrameBacktest.py:390: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'entry' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  signals[idx] = 'entry'


In [6]:
results

BacktestResult(trades=                                          exit_time  entry_price  exit_price  \
entry_time                                                                     
2024-03-21 08:00:00+00:00 2024-03-26 09:00:00+00:00        1.675      1.5100   
2024-04-02 06:00:00+00:00 2024-04-10 12:00:00+00:00        1.350      1.2300   
2024-04-12 08:00:00+00:00 2024-04-15 13:00:00+00:00        1.175      1.0600   
2024-05-10 07:00:00+00:00 2024-05-14 10:00:00+00:00        1.160      1.3199   
2024-05-16 17:00:00+00:00 2024-05-21 07:00:00+00:00        1.160      1.0767   
2024-06-03 14:00:00+00:00 2024-06-18 17:00:00+00:00        1.040      0.9800   

                           quantity    pnl    exit_type  
entry_time                                               
2024-03-21 08:00:00+00:00      1000 -165.0    stop_loss  
2024-04-02 06:00:00+00:00      1000 -120.0    stop_loss  
2024-04-12 08:00:00+00:00      1000 -115.0    stop_loss  
2024-05-10 07:00:00+00:00      1000  159.9  tak

In [7]:
class Condition(ABC):
    def __init__(self, indicator: str, comparison: str, value: Any):
        self.indicator = indicator
        self.comparison = comparison
        self.value = value
        

    @abstractmethod
    def evaluate(self, row: pd.Series) -> bool:
        pass

class IndicatorCondition(Condition):
    def __str__(self):
        return f'The indicator type is {self.indicator} and the comparison is {self.comparison} using {self.value}'

    def __repr__(self):
        return f'IndicatorCondition(\'{self.indicator}\', {self.comparison}, {self.value})'
    
    def evaluate(self, row: pd.Series) -> bool:
        # Print available columns for debugging
        print(f"\nEvaluating {self.indicator} condition:")
        print(f"Comparison: {self.comparison}")
        print(f"Value: {self.value}")
        
        # Handle special cases for indicators
        if self.indicator == "MACD" and self.value == "signal":
            macd_val = row.get('macd')
            signal_val = row.get('macdsignal')
            macd_prev = row.get('macd_prev')
            signal_prev = row.get('macdsignal_prev')
            
            print(f"MACD values:")
            print(f"Current MACD: {macd_val}")
            print(f"Current Signal: {signal_val}")
            print(f"Previous MACD: {macd_prev}")
            print(f"Previous Signal: {signal_prev}")
            
            if any(pd.isna([macd_val, signal_val, macd_prev, signal_prev])):
                print("Missing or NaN values detected")
                return False
                
            if self.comparison == "crosses_above":
                result = (macd_prev <= signal_prev) and (macd_val > signal_val)
                print(f"Crosses above evaluation: {result}")
                return result
            elif self.comparison == "crosses_below":
                result = (macd_prev >= signal_prev) and (macd_val < signal_val)
                print(f"Crosses below evaluation: {result}")
                return result
                
        elif self.indicator == "BBANDS" and self.value == "lower":
            price = row.get('close')
            lower = row.get('lowerband')  # lower band
            price_prev = row.get('close_prev')
            lower_prev = row.get('lowerband_prev')
            
            print(f"\nBBands check:")
            print(f"Current price: {price}")
            print(f"Current lower band: {lower}")
            print(f"Previous price: {price_prev}")
            print(f"Previous lower band: {lower_prev}")
            
            if any(pd.isna([price, lower, price_prev, lower_prev])):
                print("Missing or NaN values detected")
                return False
                
            if self.comparison == "crosses_below":
                result = price_prev >= lower_prev and price < lower
                print(f"Crosses below evaluation: {result}")
                return result
            elif self.comparison == "crosses_above":
                result = price_prev <= lower_prev and price > lower
                print(f"Crosses above evaluation: {result}")
                return result
                
        else:
            # Standard indicator comparison
            indicator_value = row.get(self.indicator.lower())
            if pd.isna(indicator_value):
                return False

            if isinstance(self.value, (int, float)):
                compare_value = self.value
            else:
                compare_value = row.get(self.value.lower())
                if pd.isna(compare_value):
                    return False

            if self.comparison == "above":
                print("indicator value: ", indicator_value, " / compare value: ", compare_value)
                return indicator_value > compare_value
            elif self.comparison == "below":
                print("indicator value: ", indicator_value, " / compare value: ", compare_value)
                return indicator_value < compare_value

        return False

In [8]:
def _setup_conditions(conditions_config: List[Dict]) -> List[Condition]:
    return [IndicatorCondition(
        indicator=cond['indicator'],
        comparison=cond['comparison'],
        value=cond['value']
    ) for cond in conditions_config]

def _check_conditions(conditions: List[Condition], row: pd.Series) -> bool:
    """Evaluate all conditions together and return True only if all are met"""
    print("\nChecking all conditions:")
    results = []
    for condition in conditions:
        result = condition.evaluate(row)
        results.append(result)
        print(f"- {condition.indicator} {condition.comparison} {condition.value}: {result}")
    
    all_conditions_met = all(results)
    print(f"All conditions met: {all_conditions_met}")
    return all_conditions_met



In [ ]:
entry_conditions = _setup_conditions(yaml_trade_config.get('entry_conditions'))


In [ ]:
for i, (idx, row) in enumerate(df.iterrows()):
    for condition in entry_conditions:
        if condition.evaluate(row) == False:
            print(i , idx, row.close, "no buy")
        else:
            print(i , idx, row.close, "buy")
            

In [ ]:
entry_conditions[1].evaluate(df.iloc[1000])

In [ ]:
exit_conditions = _setup_conditions(yaml_trade_config.get('exit_conditions'))
exit_conditions[0].evaluate(df.iloc[1000])

In [ ]:
exit_conditions[1].evaluate(df.iloc[1000])

In [ ]:
results.evaluation_df

In [ ]:
results.trades